# GoNuts 4

In [1]:
#load libraries
import numpy as np
from pulp import *
import pandas as pd

In [2]:
#question inputs
Products=['Ginko','Kola']
Locations=['Ethiopia','Tanzania','Nigeria']#plants
Cost_Ginko=[21,22.50,23] #in the 3 plants as above
Cost_Kola=[22.50,24.50,25.50]#in the 3 plants as above
Capacity=[425,400,750]#in the 3 plants as above
Demand=[550,450]#of the 2 drinks   

In [6]:
#lets rephrase the inputs to suit our LP modelling
Product_Demands=dict(zip(Products,Demand)) 
#using the format---> dict(zip(keys, values)) for a list keys & values
print Product_Demands
#similarly for capacity constraints
Location_Capacity=dict(zip(Locations,Capacity))
print Location_Capacity

#now we have to get the Costs for each Product and at each Location
#in such a way that we can index it easily.
adummy=[Cost_Ginko,Cost_Kola]
#below line gives first index as Products and sec index as Locations
all_combos = makeDict([Products,Locations],adummy)#this shud work
#for example
print adummy

{'Ginko': 550, 'Kola': 450}
{'Ethiopia': 425, 'Tanzania': 400, 'Nigeria': 750}
[[21, 22.5, 23], [22.5, 24.5, 25.5]]


In [26]:
#define  the problem
prob=LpProblem("nuts",LpMinimize)

#define the dec variables
x=LpVariable.dicts("dec_vars",(Products,Locations),lowBound=0,upBound=None)
#x can be accessed by x['Products'] or by x['Products']['Locations']
#example x["Kola"] gives the Kola at all 3 locations
#x["Kola"]['Nigeria'] gives Kola only at Nigeria Location

#Factory ON or OFF Decision variable
status=LpVariable.dicts("factory_status",(Locations),cat="Binary")

# The objective function is added to 'prob'
prob += lpSum([x[w][b]*all_combos[w][b] for w in Products for b in Locations])

#define the constraints
#demand cnstraint
for i in Products:
    prob +=lpSum(x[i])>=Product_Demands[i]
 
# MAXcapacity constraint
for i in Locations:
    prob +=lpSum(x[j][i] for j in Products)<=Location_Capacity[i]

#link binary dec_variables to the actual dec_Variables
for i in Locations:
    prob+=lpSum(x[j][i] for j in Products)<=[status[i]*Location_Capacity[i]]  

#only 2 plants can open
prob+=lpSum(status)<=2
    
# solve the problem
status = prob.solve()
LpStatus[status]

'Optimal'

In [27]:
# print the results
print (value(prob.objective))
print ('\n')

print(prob)
for v in prob.variables():
    print(v.name, "=", v.varValue)

22850.0


nuts:
MINIMIZE
21*dec_vars_Ginko_Ethiopia + 23*dec_vars_Ginko_Nigeria + 22.5*dec_vars_Ginko_Tanzania + 22.5*dec_vars_Kola_Ethiopia + 25.5*dec_vars_Kola_Nigeria + 24.5*dec_vars_Kola_Tanzania + 0.0
SUBJECT TO
_C1: dec_vars_Ginko_Ethiopia + dec_vars_Ginko_Nigeria
 + dec_vars_Ginko_Tanzania >= 550

_C2: dec_vars_Kola_Ethiopia + dec_vars_Kola_Nigeria + dec_vars_Kola_Tanzania
 >= 450

_C3: dec_vars_Ginko_Ethiopia + dec_vars_Kola_Ethiopia <= 425

_C4: dec_vars_Ginko_Tanzania + dec_vars_Kola_Tanzania <= 400

_C5: dec_vars_Ginko_Nigeria + dec_vars_Kola_Nigeria <= 750

_C6: dec_vars_Ginko_Ethiopia + dec_vars_Kola_Ethiopia
 - 425 factory_status_Ethiopia <= 0

_C7: dec_vars_Ginko_Tanzania + dec_vars_Kola_Tanzania
 - 400 factory_status_Tanzania <= 0

_C8: dec_vars_Ginko_Nigeria + dec_vars_Kola_Nigeria
 - 750 factory_status_Nigeria <= 0

_C9: factory_status_Ethiopia + factory_status_Nigeria
 + factory_status_Tanzania <= 2

VARIABLES
dec_vars_Ginko_Ethiopia Continuous
dec_vars_Ginko_Nigeria

In [11]:
#Factory ON or OFF Decision variable
status=LpVariable.dicts("factory_status",(Locations),cat="Binary")
print lpSum(status)

factory_status_Ethiopia + factory_status_Nigeria + factory_status_Tanzania
